# Install and import libraries

In [69]:
%pip install -q ibis-framework

Note: you may need to restart the kernel to use updated packages.


In [70]:
# Importing the required libraries
import os
import ibis
import pandas as pd
import yaml
from typing import Dict, List

## User Defined Variables

In [71]:
# Include the following tables
required_tables = ['vce_components_meta']
attr_datatypes = ['vce_datatype_lookup']

In [72]:
# Yaml file path
yaml_file_name = 'cc_db_class_info.yml'
yaml_file_path = os.path.join(os.getcwd(), yaml_file_name)

print(f"Yaml file path: {yaml_file_path}")

Yaml file path: /workspaces/coaching-companion/cc_db_class_info.yml


In [73]:
# Required Dictionaries

attr_dict = {} # Attribute dictionary (becomes a dict of dicts)
datatype_dict = {} # Data type dictionary (becomes a list of dicts)

# Functions to Access Data in the Database
## Connection Functions
### Establish the connection to the database

In [74]:
# Establish Connection to the Development Database
def gen_connection():
    conn = ibis.postgres.connect(
        host=os.environ['POSTGRES_DB_HOST'],
        user=os.environ['POSTGRES_DB_USER'],
        password=os.environ['POSTGRES_DB_PASSWORD'],
        database=os.environ['POSTGRES_DB_NAME']
    )
    return conn

## Query Functions
### Access the Table Data

In [75]:
# Get Table
def get_table(table_name, conn = gen_connection()):
    return conn.table(table_name)

### Access the Table Columns

In [76]:
# Get Table Columns
def get_table_columns(table_name, conn = gen_connection()) -> List[str]:
    table = get_table(table_name, conn)
    return table.columns

### List the Table Data

In [77]:
# List Tables
def list_tables(conn = gen_connection()) -> List[str]:
    return conn.list_tables()

### Extract the Classes Data

In [78]:
# Extract Classes
def extract_classes(conn_table,column:str = 'meta_value') -> List[str]:
    # Used when joining the same table
    a = conn_table.alias('a') # Add alias for table a
    b = conn_table.alias('b') # Add alias for table b

    # Perform the join with conditions
    joined = a.join(b, [a.component_id == b.component_id,
                        b.meta_key == 'type',
                        ~a.meta_key.like('lms_assignment_id%')])
    query = joined[b.meta_value].distinct()
    result = query.execute()
    
    return result[column].tolist()

### Extract the Attributes and Datatypes

In [79]:
def extract_attr_datatypes(conn_table1,conn_table2) -> pd.DataFrame:
    # Aliases for readability
    a = conn_table1.alias('a')
    b = conn_table1.alias('b')
    c = conn_table1.alias('c')
    d = conn_table2.alias('d')

    joined = a.join(b, a.component_id == b.component_id) \
                .join(d, b.meta_key == d.type) \
                .join(c, [a.component_id == c.component_id,
                            c.meta_key == 'type',
                            c.meta_value == 'AWSDashboard',
                            ~a.meta_key.like('lms_assignment_id%')])

    query = joined[[b.meta_key, d.datatype]].distinct()
    result = query.execute()

    return result

## Dictionary Functions
### Generate List of Datatype Dictionaries

In [80]:
def list_of_dicts(input_dataframe: pd.DataFrame, column_name_key: str, column_name_value: str) -> List[Dict[str, str]]:
    list_of_datatypes = [{row[column_name_key]: row[column_name_value]} for index, row in input_dataframe.iterrows()]
    return list_of_datatypes

### Generate Dictionary Hierarchies

In [81]:
def nested_dict(new_dict: Dict, value_dict: Dict, key) -> Dict:
    new_dict[key] = value_dict
    return new_dict

## Yaml Functions
### Create/Load the YAML File

In [82]:
def load_yaml(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

### Write the YAML File

In [83]:
def write_yaml(file_path, data):
    with open(file_path, 'w') as file:
        yaml.dump(data, file)

## Extract Database Class Data and Move to Yaml File

In [84]:
# Establish Connection to the Development Database
try:
    conn = gen_connection()
    print("Connection to the database established.")
except Exception as e:
    print(f"Error: {e}")

Connection to the database established.


In [85]:
# Verify the required tables exist in the database
tables = list_tables(conn)

if set(required_tables).issubset(set(tables)):
    print(f"All required tables found in the database.")
else:
    print(f"Table {required_tables} not found in the database.")

if set(attr_datatypes).issubset(set(tables)):
    print(f"All attribute/datatype tables found in the database.")
else:
    print(f"Table {attr_datatypes} not found in the database.")

All required tables found in the database.
All attribute/datatype tables found in the database.


In [86]:
# Loop over the required tables within the database and extract the necessary information
for i in range(len(required_tables)):

    # Initialize the dictionaries
    table_dict = {} # Table dictionary (becomes a dict of dicts)

    # Loop over the required tables
    vce_components_meta = get_table(required_tables[i],conn)
    if len(attr_datatypes) == 1:
        vce_datatype_lookup = get_table(attr_datatypes[0],conn)
    else:
        vce_datatype_lookup = get_table(attr_datatypes[i],conn)

    # Initialize the dictionaries
    class_dict = {} # Class dictionary (becomes a dict of dicts)
    # Extract the classes
    classes = extract_classes(vce_components_meta)

    # Loop over the classes
    for j in range(len(classes)):
        # Initialize the dictionaries
        attr_dict = {} # Attribute dictionary (becomes a dict of dicts)
        datatype_dict = {} # Data type dictionary (becomes a list of dicts)

        # Extract the attributes and datatypes
        attr_datatypes = extract_attr_datatypes(vce_components_meta,vce_datatype_lookup)
        
        # Loop over the attributes
        for k in range(len(attr_datatypes)):
            # Generate the list of dictionaries
            datatype_dict = list_of_dicts(attr_datatypes,'meta_key','datatype')
            # # Generate the nested dictionary
            # attr_dict = nested_dict(attr_dict,datatype_dict,attr_datatypes[k])
            
        # Generate the nested dictionary
        class_dict = nested_dict(class_dict,datatype_dict,classes[j])
        print(f"Class dictionary added the {classes[j]} class contents")

    # Create the new table in the dictionary
    table_dict = nested_dict(table_dict,class_dict,required_tables[i])
    print(f"Table dictionary added the {required_tables[i]} table contents")

Class dictionary added the Access class contents
Class dictionary added the AccreditationCertificate class contents
Class dictionary added the AccreditationSubmission class contents
Class dictionary added the ActionPlans class contents
Class dictionary added the AddPlayByPlay class contents
Class dictionary added the Alias class contents
Class dictionary added the Annotations class contents
Class dictionary added the AssessmentsClassroom class contents
Class dictionary added the AssessmentsDashbaord class contents
Class dictionary added the AssessmentsEvaluation class contents
Class dictionary added the AssessmentsLister class contents
Class dictionary added the AssessmentsRating class contents
Class dictionary added the AssessmentsReport class contents
Class dictionary added the AssessmentsReportClassroom class contents
Class dictionary added the AssessmentsSite class contents
Class dictionary added the Assets class contents
Class dictionary added the Assignments class contents
Class 

In [87]:
# Verify contents of the table dictionary
print("The table dictionary is as follows:")
print(yaml.dump(table_dict))

The table dictionary is as follows:
vce_components_meta:
  AWSDashboard:
  - created_by: int
  - created_at: unix timestamp
  - title: varchar(255)
  - type: varchar(50)
  Access:
  - created_by: int
  - created_at: unix timestamp
  - title: varchar(255)
  - type: varchar(50)
  AccreditationCertificate:
  - created_by: int
  - created_at: unix timestamp
  - title: varchar(255)
  - type: varchar(50)
  AccreditationSubmission:
  - created_by: int
  - created_at: unix timestamp
  - title: varchar(255)
  - type: varchar(50)
  ActionPlans:
  - created_by: int
  - created_at: unix timestamp
  - title: varchar(255)
  - type: varchar(50)
  AddPlayByPlay:
  - created_by: int
  - created_at: unix timestamp
  - title: varchar(255)
  - type: varchar(50)
  Alias:
  - created_by: int
  - created_at: unix timestamp
  - title: varchar(255)
  - type: varchar(50)
  Annotations:
  - created_by: int
  - created_at: unix timestamp
  - title: varchar(255)
  - type: varchar(50)
  AssessmentsClassroom:
  - cr

In [88]:
# Write the dictionary to a yaml file
write_yaml(yaml_file_path,table_dict)
print(f"Yaml file written to {yaml_file_path}")

Yaml file written to /workspaces/coaching-companion/cc_db_class_info.yml
